In [25]:
import pandas as pd
import numpy as np 
import ast

bank_churn = pd.read_csv('/Users/kyleleighton/Documents/Bank_churn.csv')
print(bank_churn.head())

    id  CustomerId                Surname  CreditScore   Age  Tenure  \
0  0.0  15674932.0   O,k{wudilic_-huk>wu_        668.0  33.0     3.0   
1  1.0  15749177.0  O{kwudi"li#%%'oli=-sa        627.0  33.0     1.0   
2  2.0  15694510.0                  Hsueh        678.0  40.0    10.0   
3  3.0  15741417.0            Kao,]?>?%]@        581.0  34.0     2.0   
4  4.0  15766172.0          Ch-i]emen'-am        716.0  33.0     5.0   

     Balance  NumOfProducts  HasCrCard  IsActiveMember EstimatedSalary  \
0       0.00            2.0        1.0             0.0       181449.97   
1       0.00            2.0        1.0             1.0         49503.5   
2       0.00            2.0        1.0             0.0       184866.69   
3  148882.54            1.0        1.0             1.0        84560.88   
4       0.00            2.0        1.0             1.0        15068.83   

   Rating  Exited                        Geography:str,Gender:str  
0      -3       0  {'Geography': 'Paris-France','Gende

In [26]:
#separating column 'Geograph:str,Gender:str' into 2 columns, 'Geography' and 'Gender'
bank_churn['GeographyGender'] = bank_churn['Geography:str,Gender:str'].apply(ast.literal_eval)
bank_churn[['Geography', 'Gender']] = bank_churn['GeographyGender'].apply(pd.Series)
bank_churn = bank_churn.drop(['GeographyGender','Geography:str,Gender:str'], axis=1)
print(bank_churn.head())

    id  CustomerId                Surname  CreditScore   Age  Tenure  \
0  0.0  15674932.0   O,k{wudilic_-huk>wu_        668.0  33.0     3.0   
1  1.0  15749177.0  O{kwudi"li#%%'oli=-sa        627.0  33.0     1.0   
2  2.0  15694510.0                  Hsueh        678.0  40.0    10.0   
3  3.0  15741417.0            Kao,]?>?%]@        581.0  34.0     2.0   
4  4.0  15766172.0          Ch-i]emen'-am        716.0  33.0     5.0   

     Balance  NumOfProducts  HasCrCard  IsActiveMember EstimatedSalary  \
0       0.00            2.0        1.0             0.0       181449.97   
1       0.00            2.0        1.0             1.0         49503.5   
2       0.00            2.0        1.0             0.0       184866.69   
3  148882.54            1.0        1.0             1.0        84560.88   
4       0.00            2.0        1.0             1.0        15068.83   

   Rating  Exited     Geography Gender  
0      -3       0  Paris-France   Male  
1       0       0  Paris-France   Male  

In [27]:
#removing 'k' from instances where estimated salary  includes values like '50k'
bank_churn['EstimatedSalary'] = bank_churn['EstimatedSalary'].astype(str).str.replace('k', '', regex = False).astype(float)
#removing anything that is not a letter from surnames
bank_churn['Surname'] = bank_churn['Surname'].astype(str).str.replace(r'[^A-Z, a-z]', '', regex = True).str.replace(',', '').dropna().astype(str)
#removing instances in age where it includes values like '35 years old' 
bank_churn['Age'] = pd.to_numeric(bank_churn['Age'].astype(str).str.replace(r'[^0-9.]', '', regex = True), errors = 'coerce')
#stripping extra zeros on the end of ages
def clean_age(x):
    if pd.isnull(x) or x <= 120:
        return int(x) if not pd.isnull(x) else x
    # Reduce number until it’s <= 120
    while x > 120:
        x = x // 10
    return int(x)

bank_churn['Age'] = bank_churn['Age'].apply(clean_age)
#removing 'k' from instances where balance includes values like '50k'
bank_churn['Balance'] = bank_churn['Balance'].astype(str).str.replace('k', '', regex = False).astype(float)
#convert 'Tenure' column to numeric; invalid values will become NaN instead of raising an error
bank_churn['Tenure'] = pd.to_numeric(bank_churn['Tenure'], errors='coerce')
# standardize 'Rating' by converting all values to non-negative (absolute values) 
bank_churn['Rating'] = bank_churn['Rating'].astype(int).abs()
# and cast to an ordered categorical type (0–5) for consistent analysis and modeling
bank_churn['Rating'] = pd.Categorical(bank_churn['Rating'], categories=[0, 1, 2, 3, 4, 5], ordered=True)
#replace all empty strings with NaN values
bank_churn = bank_churn.replace('', np.nan, regex = False)
#drop all rows with NaN values from bank_churn
bank_churn = bank_churn.dropna()
print(bank_churn.head())

    id  CustomerId         Surname  CreditScore   Age  Tenure    Balance  \
0  0.0  15674932.0  Okwudilichukwu        668.0  33.0     3.0       0.00   
1  1.0  15749177.0   Okwudiliolisa        627.0  33.0     1.0       0.00   
2  2.0  15694510.0           Hsueh        678.0  40.0    10.0       0.00   
3  3.0  15741417.0             Kao        581.0  34.0     2.0  148882.54   
4  4.0  15766172.0       Chiemenam        716.0  33.0     5.0       0.00   

   NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary Rating  Exited  \
0            2.0        1.0             0.0        181449.97      3       0   
1            2.0        1.0             1.0         49503.50      0       0   
2            2.0        1.0             0.0        184866.69      1       0   
3            1.0        1.0             1.0         84560.88      4       0   
4            2.0        1.0             1.0         15068.83      1       0   

      Geography Gender  
0  Paris-France   Male  
1  Paris-France   

In [28]:
#correcting Germany name and removing rows where value is 'nan' string
bank_churn['Geography'] = bank_churn['Geography'].replace('Berlin-Gernay', 'Berlin-Germany')
bank_churn = bank_churn[~bank_churn.isin(['nan']).any(axis=1)]
print(bank_churn['Geography'].unique())

['Paris-France' 'Madrid-Spain' 'Berlin-Germany']


In [29]:
#setting columns to the appropriate types
bank_churn['Tenure'] = bank_churn['Tenure'].astype(int)
bank_churn['Age'] = bank_churn['Age'].astype(int)
bank_churn['id'] = bank_churn['id'].astype(int)
bank_churn['CustomerId'] = bank_churn['CustomerId'].astype(int)
bank_churn['CreditScore'] = bank_churn['CreditScore'].astype(int)
bank_churn['IsActiveMember'] = bank_churn['IsActiveMember'].astype(bool)
bank_churn['Exited'] = bank_churn['Exited'].astype(bool)
bank_churn['NumOfProducts'] = bank_churn['NumOfProducts'].astype(int)
bank_churn['HasCrCard'] = bank_churn['HasCrCard'].astype(bool)
bank_churn['Geography'] = bank_churn['Geography'].astype('string')
bank_churn['Gender'] = bank_churn['Gender'].astype('string')
bank_churn.dtypes

id                          int64
CustomerId                  int64
Surname                    object
CreditScore                 int64
Age                         int64
Tenure                      int64
Balance                   float64
NumOfProducts               int64
HasCrCard                    bool
IsActiveMember               bool
EstimatedSalary           float64
Rating                   category
Exited                       bool
Geography          string[python]
Gender             string[python]
dtype: object

In [30]:
# Save the cleaned DataFrame to a CSV file
csv_path = ('/Users/kyleleighton/Documents/Bank_churn_cleaned.csv')
bank_churn.to_csv(csv_path, index=False)
bank_churn_cleaned = pd.read_csv('/Users/kyleleighton/Documents/Bank_churn_cleaned.csv')